# TCP Socket application with MicroK8s on Ubuntu 20.04

"MicroK8s is a powerful, lightweight, reliable production-ready Kubernetes distribution. It is an enterprise-grade Kubernetes distribution that has a small disk and memory footprint while offering carefully selected add-ons out-the-box, such as Istio, Knative, Grafana, Cilium and more. Whether you are running a production environment or interested in exploring K8s, MicroK8s serves your needs," see [Introduction to MicroK8s](https://ubuntu.com/blog/introduction-to-microk8s-part-1-2).

### The model

AccountReceivable and GeneralLedger are microservices. We use tcp socket protocol for communication.

To create statefulset pods we used Suyash Mohan article [Setting up PostgreSQL Database on Kubernetes](https://medium.com/@suyashmohan/setting-up-postgresql-database-on-kubernetes-24a2a192e962) as a guideline.
```
                         Store
                           ↕
                       Counter (cnt)
                           ↓
    testpluto.jl        Invoice Nbr
          ↓                ↓
      Orders/ → AccountsReceivable (ar) → Entries → GeneralLedger (gl)
  BankStatements           ↕                               ↕
                         Store                           Store
                  Unpaid/PaidInvoices            Journal/GeneralLedger
```

### Check whether ar-statefulset, gl-statefullset and cnt-statefullset pods are running

Run the next command from the terminal:

**microk8s.kubectl get pods -n socket-ns**

```
NAME                READY   STATUS    RESTARTS   AGE
cnt-statefulset-0   2/2     Running   0          160m
gl-statefulset-0    2/2     Running   0          159m
gl-statefulset-1    2/2     Running   0          159m
ar-statefulset-1    2/2     Running   0          98m
ar-statefulset-0    2/2     Running   0          97m
```

### If pods are not running use next commands from the terminal
- Install microk8s: **sudo snap install microk8s --classic**
- Enable microk8s build-in apps: **microk8s enable registry dashboard dns istio storage**
- Clone files from GitHub: **git clone https://github.com/rbontekoe/ar.git**
- Enter the folder with the downloaded files: **cd ar**
- Download Julia 1.6.5: **curl -O https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.5-linux-x86_64.tar.gz**
- Create accounts receivable image: **docker build --no-cache -f ar.Dockerfile -t localhost:32000/i_ar:v1.0.16 .**
- Copy to local registry: **docker push localhost:32000/i_ar:v1.0.16**
- Create general ledger image: **docker build --no-cache -f gl.Dockerfile -t localhost:32000/i_gl:v1.0.4 .**
- Copy to local registry: **docker push localhost:32000/i_gl:v1.0.4**
- Create counter image: **docker build --no-cache -f cnt.Dockerfile -t localhost:32000/i_cnt:v1.0.1 .**
- Copy to local registry: **docker push localhost:32000/i_cnt:v1.0.1**
- **microk8s.kubectl apply -f 1-ar-storage.yaml**
- **microk8s.kubectl apply -f 2-gl-storage.yaml**
- **microk8s.kubectl apply -f 3-cnt-storage.yaml**

## Connect to AppliAR.jl, create and process the orders

In [9]:
import Pkg; Pkg.add(url="https://github.com/rbontekoe/AppliAR")

### Load the files

In [10]:
using Sockets, Serialization, AppliSales, AppliAR, AppliGeneralLedger, DataFrames, Query

### Delete the old data files

Go to the terminal and delete the next files if you want a fresh start:

```
sudo rm -f generalledger.txt paid-invoices.txt unpaid-invoices.txt journal.txt seqnbr.txt
```

### Connect to AppliAR.jl, create and process the orders

In [28]:
clientside = connect(ip"127.0.0.1", 30012) # connect to accounts receivable pod

TCPSocket(RawFD(49) open, 0 bytes waiting)

In [12]:
data = ""
@async while isopen(clientside)
    global data = deserialize(clientside)
end

Task (runnable) @0x00007f063d955660

In [16]:
sales = AppliSales.process() # create sales orders
serialize(clientside, sales) # send orders to account receivable

In [17]:
serialize(clientside, "gl-status")
DataFrame(data)

,id,accountid,date,customerid
,String,Int64,Dates…,String
1,2022-04-12-1001,1300,"DateTime(""2022-04-12T14:49:05.995"")",Scrooge Investment Bank
2,2022-04-12-1001,8000,"DateTime(""2022-04-12T14:49:05.995"")",Scrooge Investment Bank
3,2022-04-12-1001,4000,"DateTime(""2022-04-12T14:49:05.995"")",Scrooge Investment Bank
4,2022-04-12-1002,1300,"DateTime(""2022-04-12T14:49:05.995"")",Duck City Chronicals
5,2022-04-12-1002,8000,"DateTime(""2022-04-12T14:49:05.995"")",Duck City Chronicals
6,2022-04-12-1002,4000,"DateTime(""2022-04-12T14:49:05.995"")",Duck City Chronicals
7,2022-04-12-1003,1300,"DateTime(""2022-04-12T14:49:05.995"")",Donalds Hardware Store
8,2022-04-12-1003,8000,"DateTime(""2022-04-12T14:49:05.995"")",Donalds Hardware Store
9,2022-04-12-1003,4000,"DateTime(""2022-04-12T14:49:05.995"")",Donalds Hardware Store


### Load and process the bank statements

In [18]:
stms = AppliAR.read_bank_statements("./bank-kubernetes.csv") # retrieve the bankstatements

2-element Vector{BankStatement}:
 BankStatement(Dates.Date("2020-01-15"), "Duck City Chronicals Invoice 1002", "NL93INGB", 2420.0)
 BankStatement(Dates.Date("2020-01-15"), "Donalds Hardware Store Bill 1003", "NL39INGB", 1210.0)

In [19]:
serialize(clientside, stms) # create paid invoices and update general ledger

### Display Accounts Receivable

Other accounts are:

```
8000 - Sales
1150 - Bank
4000 - VAT
1300 - Accounts Receivable
```

In [24]:
accountid = 1300;

In [25]:
#r2 = AppliGeneralLedger.read_from_file("/var/data-ar/generalledger.txt")
#df2 = r2 |> @filter(_.accountid == accountid) |> DataFrame
#df2[:, [:accountid, :customerid, :invoice_nbr, :debit, :credit, :descr]]
serialize(clientside, "gl-status");
df2 = data |> @filter(_.accountid == accountid) |> DataFrame
df2[:, [:accountid, :customerid, :invoice_nbr, :debit, :credit, :descr]]

,accountid,customerid,invoice_nbr,debit,credit,descr
,Int64,String,String,Float64,Float64,String
1,1300,Scrooge Investment Bank,1001,1210.0,0.0,Learn Smiling
2,1300,Duck City Chronicals,1002,2420.0,0.0,Learn Smiling
3,1300,Donalds Hardware Store,1003,1210.0,0.0,Learn Smiling
4,1300,Duck City Chronicals,1002,0.0,2420.0,Learn Smiling
5,1300,Donalds Hardware Store,1003,0.0,1210.0,Learn Smiling
6,1300,Scrooge Investment Bank,1004,1210.0,0.0,Learn Smiling
7,1300,Duck City Chronicals,1005,2420.0,0.0,Learn Smiling
8,1300,Donalds Hardware Store,1006,1210.0,0.0,Learn Smiling
9,1300,Scrooge Investment Bank,1007,1210.0,0.0,Learn Smiling


### Display the status of the unpaid invoices

In [33]:
serialize(clientside, "status");

In [34]:
DataFrame(data)

,id,accountid,date,customerid
,String,Int64,Dates…,String
1,2022-04-12-1001,1300,"DateTime(""2022-04-12T14:49:05.995"")",Scrooge Investment Bank
2,2022-04-12-1001,8000,"DateTime(""2022-04-12T14:49:05.995"")",Scrooge Investment Bank
3,2022-04-12-1001,4000,"DateTime(""2022-04-12T14:49:05.995"")",Scrooge Investment Bank
4,2022-04-12-1002,1300,"DateTime(""2022-04-12T14:49:05.995"")",Duck City Chronicals
5,2022-04-12-1002,8000,"DateTime(""2022-04-12T14:49:05.995"")",Duck City Chronicals
6,2022-04-12-1002,4000,"DateTime(""2022-04-12T14:49:05.995"")",Duck City Chronicals
7,2022-04-12-1003,1300,"DateTime(""2022-04-12T14:49:05.995"")",Donalds Hardware Store
8,2022-04-12-1003,8000,"DateTime(""2022-04-12T14:49:05.995"")",Donalds Hardware Store
9,2022-04-12-1003,4000,"DateTime(""2022-04-12T14:49:05.995"")",Donalds Hardware Store
